# GeoPandas

GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting.

The goal of GeoPandas is to make working with geospatial data in python easier. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS.

## OGC SimpleFeatures

A standard that specifies a common storage and access model of mostly two-dimensional geographical data (point, line, polygon, multi-point, multi-line, etc.) The formats were originally defined by the Open Geospatial Consortium (OGC) and described in their Simple Feature Access and Coordinate Transformation Service specifications. 

The standard defines a model for two-dimensional simple features, with linear interpolation between vertices. The data model defined is a hierarchy of classes. This part also defines representation using Well-Known Text (and Binary).

### WKT/WKB: Well Known Text and Well Known Binary

Well-known text (WKT) is a text markup language for representing vector geometry objects on a map, spatial reference systems of spatial objects and transformations between spatial reference systems. A binary equivalent, known as well-known binary (WKB), is used to transfer and store the same information on databases, such as PostGIS, Microsoft SQL Server and DB2. The formats were originally defined by the Open Geospatial Consortium (OGC) and described in their Simple Feature Access and Coordinate Transformation Service specifications.


### Implications for PySpark data de/serialization & un/marshalling

When using pyspark we want have to send data back and forth between master node and the workers which run jobs on the JVM. In order to simplify this rather than sending Python or more precisely Shapely objects we will use WKT. It works with one the libraries I am using today to perform Spatial Joins.


In [ ]:
%load './code/helpers/imports.py'

In [ ]:
%load './code/helpers/osm2geojson.py'

In [ ]:
%load './code/helpers/query_overpass_api.py'

In [ ]:
OVERPASS_API         = overpy.Overpass()
BASE_DIR             = os.path.join(os.path.abspath('.'), 'work-flow')
URBAN_BOUNDARIES_FILE = '06_Europe_Cities_Boundaries_with_Labels_Population.geo.json'

# Paths to base datasets that we are using:
URBAN_BOUNDARIES_PATH = os.path.join(BASE_DIR,URBAN_BOUNDARIES_FILE)
POIS_PATH            = os.path.join(BASE_DIR, "pois.json")

In [ ]:
# Load the boundaries data
geo_df = GeoDataFrame.from_file(URBAN_BOUNDARIES_PATH)
geo_df.plot()

In [ ]:
# Fetch the the POIs but not now
overwritePois = False # <= DON'T Set to True during workshop!        
if overwritePois or not os.path.isfile(POIS_PATH):
    # Write POIs file
    pois = run_overpass_api(geo_df)
    with io.open(POIS_PATH, 'w+', encoding='utf-8') as f:
        f.write(unicode(json.dumps(pois, ensure_ascii= False)))

For the exercise below consult GeoPandas documentation http://geopandas.org/user.html

## Exercise: 

* find out what the schema for the data is?
* plot the geometries?
* print out the wkt version of the geometries
* find out the Spatial Reference System (CRS/SRS) for the GeoDataframe?
* change the reference system to a cartesian system that is suitable for calculating areas
* calculate the total area of the cities we are using HINT: Europe using this CRS: http://spatialreference.org/ref/epsg/3035/

In [ ]:
# Answer

# * find out what the schema for the data is?

# * plot the geometries?

# * print out the wkt version of the geometries

# * find out the Spatial Reference System (CRS/SRS) for the GeoDataframe?
# A note about CRS's and Geospatial Data.
# http://spatialreference.org/ref/epsg/etrs89-etrs-laea/

# * change the reference system to a cartesian system that is suitable for calculating areas
# * calculate the total area of the cities we are using 
#   HINT: Europe using this CRS: http://spatialreference.org/ref/epsg/3035/


In [ ]:
## Solution

# * find out what the schema for the data is?
geo_df

In [ ]:
# * plot the geometries?
plt.figure(figsize=(18, 20))
geo_df.plot()

In [ ]:
# * print out the wkt version of the geometries
wkts = map(lambda g: g.to_wkt() , geo_df.geometry)
wkts[0]

In [ ]:
# * find out the Spatial Reference System (CRS/SRS) for the GeoDataframe?
# A note about CRS's and Geospatial Data.
# http://spatialreference.org/ref/epsg/etrs89-etrs-laea/
geo_df.crs

In [ ]:
# * change the reference system to a cartesian system that is suitable for calculating areas
# * calculate the total area of the cities we are using 
#   HINT: Europe using this CRS: http://spatialreference.org/ref/epsg/3035/
geo_df_cartesian = geo_df.to_crs(epsg=3035)
print geo_df_cartesian.crs
print geo_df.crs
geo_df_cartesian.area.sum()/1000000

## Preparing our data for Spatial Spark

The way I have written spatial spark is that it expects a geospatial column as a WKT string. Internally it uses this to create OGC Geometries via Java Topology Suite (JTS). So in order to use Spatial Spark we will add the WKT column to our data.

In [ ]:
# Add a WKT column for use later
geo_df['wkt'] = pandas.Series(
    map(lambda geom: str(geom.to_wkt()), geo_df['geometry']), 
    index=geo_df.index, dtype='string')

In [ ]:
geo_df['wkt']

## Create a pyspark dataframe from GeoPanadas

Spark Dataframes are an abstraction over RDDs. Lets look at what happens when we call pyspark transform and action. 

**SLIDES Switch: Spark Internals**

![](images/pyspark.png)

In [ ]:
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.createDataFrame
# Creates a DataFrame from an RDD of tuple/list, list or pandas.DataFrame.

# When schema is a list of column names, the type of each column will be inferred from data.

# When schema is None, it will try to infer the schema (column names and types) from data, which should
# be an RDD of Row, or namedtuple, or dict.

# If schema inference is needed, samplingRatio is used to determined the ratio of rows used for schema inference.
# The first row will be used if samplingRatio is None.

# Parameters:
# data – an RDD of Row/tuple/list/dict, list, or pandas.DataFrame.
# schema – a StructType or list of column names. default None.
# samplingRatio – the sample ratio of rows used for inferring
# Returns:
# DataFrame

boundaries_from_pd = sqlContext.createDataFrame(geo_df)

In [ ]:
# Lets find Berlin
berlin_rdd = boundaries_from_pd.filter(boundaries_from_pd['NAMEASCII'] == 'Berlin')
wkt.loads(berlin_rdd.take(1)[0].wkt)

### Exercise 

* Look at the schema of the boundaries_from_pd
* Graph the name of the city and its population sorted

In [ ]:
# Answer 10 min

# 1. print schema

# 2. Print of cities sorted by its population
# Hint you can go from spark data fram to Pandas data frame and plot that using the pandas plot function

In [ ]:
# -------------------------- Don't look Below untill you have tried the Exerise ----------------------------- #










# Solution:
boundaries_from_pd.printSchema()


df = boundaries_from_pd.select(boundaries_from_pd['NAMEASCII'], 
                              boundaries_from_pd['POPEU2013'].cast(IntegerType())
                             )

df = df.sort(df['POPEU2013'].desc())
df.show()

# # SQL Version
boundaries_from_pd.registerTempTable('boundaries')
sqlContext.sql("SELECT NAME, POPEU2013 FROM boundaries").show()


In [ ]:
df = df.toPandas()
df.plot.bar(x='NAMEASCII')

# Lets look at the POI data next

Our POI data comes from Openstreetmap and its stored as lines of GeoJSON point features. This format is perfect for working with Apache Spark. Spark loads the data from each row and infers the schema as well. This data could be stored on Amazon S3 or Hadoop File System (HDFS) allowing for relative huge datasets. Even though in our example we are reading the JSON from file we can just as easily point it to HDFS.

In [ ]:
pois_df = sqlContext.read.json(POIS_PATH)

print pois_df.count()
rec = pois_df.take(1)[0]


## Adding WKT attribute

Similar to the data for boundaries we want to add WKT column. Notice how we can go from Spark Dataframe to Pandas dataframes and back seamlessly.

In [ ]:
def toWktColumn(coords):
    return (Point(coords).wkt)

pois_df = pois_df.toPandas()
pois_df['wkt'] = pandas.Series(
    map(lambda geom: toWktColumn(geom.coordinates), pois_df['geometry']), 
    index=pois_df.index, dtype='string')

pois_df = sqlContext.createDataFrame(pois_df)